In [1]:
import numpy as np
import torch
import pandas as pd
from model.util import Normalizer
from model.database_util import get_hist_file, get_job_table_sample, collator
from model.model import QueryFormer
from model.dataset import PlanTreeDataset
from model.util import seed_everything

%load_ext autoreload
%autoreload 2

In [2]:
class Args:
    pass

data_path = './data/imdb/'
hist_file = get_hist_file(data_path + 'histogram_string.csv')
cost_norm = Normalizer(-3.61192, 12.290855)
encoding_ckpt = torch.load('checkpoints/encoding.pt')
encoding = encoding_ckpt['encoding']
checkpoint = torch.load('checkpoints/cost_model.pt', map_location='cpu')
seed_everything()
args = checkpoint['args']

/Users/rheinrich/Projekte/QueryFormer/model/database_util.py:76: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  hist_file['freq'][i] = freq_np
/Users/rheinrich/Projekte/QueryFormer/model/database_util.py:89: FutureWarning: ChainedAssignmentEr

In [4]:
model = QueryFormer(emb_size = args.embed_size,
                    ffn_dim = args.ffn_dim, 
                    head_size = args.head_size,
                    dropout = args.dropout, 
                    n_layers = args.n_layers,
                    use_sample = True, 
                    use_hist = True,
                    pred_hid = args.pred_hid)
model.load_state_dict(checkpoint['model'])
device = 'cpu'
_ = model.to(device).eval()
to_predict = 'cost'
methods = {
    'get_sample' : get_job_table_sample,
    'encoding': encoding,
    'cost_norm': cost_norm,
    'hist_file': hist_file,
    'model': model,
    'device': device,
    'bs': 512,
}

In [5]:
def print_qerror(preds_unnorm, labels_unnorm):
    qerror = []
    for i in range(len(preds_unnorm)):
        if preds_unnorm[i] > float(labels_unnorm[i]):
            qerror.append(preds_unnorm[i] / float(labels_unnorm[i]))
        else:
            qerror.append(float(labels_unnorm[i]) / float(preds_unnorm[i]))

    e_50, e_90 = np.median(qerror), np.percentile(qerror,90)    
    e_mean = np.mean(qerror)
    print("Median: {}".format(e_50))
    print("90th percentile: {}".format(e_90))
    print("Mean: {}".format(e_mean))
    return 

def evaluate(model, ds, bs, norm, device):
    model.eval()
    cost_predss = np.empty(0)

    with torch.no_grad():
        for i in range(0, len(ds), bs):
            batch, batch_labels = collator(list(zip(*[ds[j] for j in range(i,min(i+bs, len(ds)) ) ])))
            batch = batch.to(device)
            cost_preds, _ = model(batch)
            cost_preds = cost_preds.squeeze()
            cost_predss = np.append(cost_predss, cost_preds.cpu().detach().numpy())
    print_qerror(norm.unnormalize_labels(cost_predss), ds.costs)
    return

def eval_workload(workload, methods):
    get_table_sample = methods['get_sample']
    workload_file_name = './data/imdb/workloads/' + workload
    table_sample = get_table_sample(workload_file_name)
    plan_df = pd.read_csv('./data/imdb/{}_plan.csv'.format(workload))
    workload_csv = pd.read_csv('./data/imdb/workloads/{}.csv'.format(workload),sep='#',header=None)
    workload_csv.columns = ['table','join','predicate','cardinality']
    
    ds = PlanTreeDataset(plan_df, workload_csv,
                         methods['encoding'], methods['hist_file'], methods['cost_norm'],
                         methods['cost_norm'], 'cost', table_sample)

    evaluate(methods['model'], ds, methods['bs'], methods['cost_norm'], methods['device'])
    return 

In [6]:
eval_workload('job-light', methods)

Loaded queries with len  70
Loaded bitmaps


/Users/rheinrich/Projekte/QueryFormer/model/dataset.py:109: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  'features' : torch.FloatTensor(features),


Median: 1.4075485927633589
90th percentile: 20.276218730998703
Mean: 12.49416464740001


In [7]:
eval_workload('synthetic', methods)

Loaded queries with len  5000
Loaded bitmaps
Median: 1.065068244934082
90th percentile: 1.7178602990562961
Mean: 1.5365826878629782
